In [ ]:
#단어 모델을 학습하여 다음에 나올 단어를 예측하는 걸 만들자!

%pylab
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,tensorflow

In [ ]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)


In [ ]:
import time
import tensorflow as tf
from tensorflow.contrib.rnn import BasicLSTMCell, MultiRNNCell, DropoutWrapper

import reader

In [ ]:
#RNN에 쓰이는 설정들을 모아서 간단한 클래스를 만들자!
# smallConfig, MediumConfig, Large,,,TestConfig가 있다
# TestConfig는 모델을 검증하거나 테스트할 때 사용하고 GPU를 사용하지 않는 개인 컴퓨터에는 smallConfig을 쓰자
class SmallConfig(object):
    init_scale =0.1 # 가중치 행렬을 랜덤하게 초기화 할 때 생성되는 값의 범위 지정

    learning_rate = 1.0 # 경사하강법에서 학습 속도를 조절하기 위한 변수 
    lr_decay = 0.5 # 학습속도 = 학습률 * lr_decay, 학습이 반복될때마다 lr_decay는 작아짐
                   # 소수이므로 제곱할수록 작아지기 때문
                   # 즉, 초기에는 학습속도 빠르게 오차함수를 이동하지만, 점차 천천히 최적값에 수렴하게 됨.

    max_grad_norm = 5 # 구해진 기울기(gradient)값이 과다하게 커지는것을 막는 상한 값 cf.기울기 clipping
    
    num_layers = 2 # RNN을 구성할 계층의 개수
    hidden_size = 200 #한계층에 배치할 뉴런(셀)의 개수 -> 총 400뉴런 있게 됨

    num_steps = 20 # RNN을 사용하여 연속적으로 처리할 데이터의 양
                   # 즉 이 횟수만큼 셀의 가중치를 학습시킨 후, (경사하강법으로) 기울기을 업데이트 할 것.

    max_max_epoch = 13 #전체학습은 13회 반복 
    max_epoch = 4 # 그중에서 max_epoch 값이 다 커질때까지(5회까지) 초기 학습 속도 유지
    
    keep_prob = 1.0 # 드롭아웃 하지 않을 확률 즉 여기 small 에서는 드롭아웃을 사용하지 않는다
    batch_size = 20 # 이 예제에서는 한번의 학습에 전체 데이터를 사용하지 않고 일부만 사용하는 미니 배치 법 사용
                    # 20은 이 미니 batch 의 크기
    vocab_size = 10000 # 주어진 학습 데이터 ptb.train.txt 에는 고유한 단어가 1만개 들어있다. 이를 지정해줌.
    
# 미리 단어 전처리에 대해서 언급
# 단어 횟수가 높은 순으로 정렬 후 차례대로 번호 부여함.

# ptb_iterator
# 또한 학습 데이터를 배치 개수로 나누어 num_step 의 크기 만큼 나누어 읽어옴
# 전체 학습 데이터를 batch size인 20개로 나누면 각 배치의 크기느 00000개가 되므로, [20, 00000]의 2차원 배열만든다
# ptb_iterator는 호출될 때마다 이 2차원 배열에서 num_step 크기인 20개씩 읽어서 미니배치용 학습 데이터로 리턴. 
    

In [ ]:
#트레이닝과 테스트에 사용할 두개의 config 오브젝트를 만듭니다

config = SmallConfig()
eval_config = SmallConfig()

eval_config.batch_size = 1
eval_config.num_steps = 1

In [ ]:
#PTB 모델을 만들어 주는 클래스를 작성합니다.
class PTBModel(object):
    """The PTB model."""
    # init에서 텐서플로를 사용하여 신경망 모델을 모두 구성
    def __init__(self, config, is_training=False):
        self.batch_size = config.batch_size
        self.num_steps = config.num_steps
        input_size = [config.batch_size, config.num_steps]
        self.input_data = tf.placeholder(tf.int32, input_size)
        self.targets = tf.placeholder(tf.int32, input_size)
        
        
        # 텐서플로에서 제공하는 RNN 클래스 중 BasicLSTMCell 클래스를 이용
        lstm_fn = lambda: BasicLSTMCell(config.hidden_size, forget_bias=0.0, state_is_tuple=True, 
                                        reuse=tf.get_variable_scope().reuse)
        #여기서 state_is_tuple = true로 했는데, 이는 성능 향상을 위해 셀/은닉 상태 데이터를 튜플로 관리하게 한다.
        
        # SmallConfig에서는 드롭아웃이 적용되지 않습니다.
        if is_training and config.keep_prob < 1:
            lstm_fn = lambda: DropoutWrapper(lstm_fn(), config.keep_prob)
        
        # 두개의 계층을 가진 신경망 구조를 만듭니다.
        cell = MultiRNNCell([lstm_fn() for _ in range(config.num_layers)], state_is_tuple=True)

        self.initial_state = cell.zero_state(config.batch_size, tf.float32)
    
        # 모델에 입력되는 학습 데이터는 word to idx 한 [20,20] 배열 데이터. 
        # 숫자들로 단어가 나타나졌을 때, 단어의 벡터표현(word2vec)을 하기 위해서 
        # 단어 임베딩 작업을 수행한다.
        with tf.device("/cpu:0"):
            embedding_size = [config.vocab_size, config.hidden_size]
            embedding = tf.get_variable("embedding", embedding_size)
            inputs = tf.nn.embedding_lookup(embedding, self.input_data)
        # 그래서 최종적으로 입력데이터 input은 최종적으로 [20,20,200] 행렬이 됩.
        # [20개로 나눈 배치의 개수, num_steps만큼 각 배치에서 읽어온 데이터, 벡터값?]
        # SmallConfig에서는 드롭아웃이 적용되지 않습니다.
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)
            
        # 입력 데이터를 한 스텝씩 셀에 주입하는 부분
        # 각 batch 마다 순서대로 데이터를 뽑아 셀에 입력합니다. 
        outputs = []
        state = self.initial_state
        with tf.variable_scope("RNN"):
            # num_steps 값만큼 입력 데이터를 순회하면서 셀 객체를 실행
            for time_step in range(config.num_steps): 
                if time_step > 0: tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                outputs.append(cell_output)
            # 결국 cell에 입력되는 최종 데이터는 [20,200] 배열이 된다.
                

        # output의 크기를 20x20x200에서 400x200으로 변경합니다.
        output = tf.reshape(tf.concat(outputs, 1), [-1, config.hidden_size])
        softmax_w_size = [config.hidden_size, config.vocab_size]
        
        softmax_w = tf.get_variable("softmax_w", softmax_w_size)
        #소프트 맥스 계층의 가중치 행렬인 softmax_w 는 [200,10000]
        softmax_b = tf.get_variable("softmax_b", [config.vocab_size])
        #편향 값인 b는 크기 [10000]의 행 벡터
        
        # 그래서 결국 소프트 맥스 계층에서 계산되는 logits의 크기는 400x10000이 됩니다.
        logits = tf.matmul(output, softmax_w) + softmax_b
        
        # sequence_loss_by_example -> 소프트 맥스 함수와 크로스 엔트로피 계산을 처리
        loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example(
            [logits],
            [tf.reshape(self.targets, [-1])],
            [tf.ones([config.batch_size * config.num_steps])])
        self.cost = tf.reduce_sum(loss) / config.batch_size
        self.final_state = state

        if not is_training:
            return

        self.lr = tf.Variable(0.0, trainable=False)
        tvars = tf.trainable_variables()
        
        # 기울기 클리핑을 수행합니다. config.max_grad_norm을 기준으로 클리핑
        # 경사 하강법 최적화 클래스 객체를 형성해서 모델으 ㅣ매개변수들을 학습 시킨다.
        # 여기서 학습되는 것은 [10000,200]크기의 단어 임베딩 행렬
        # 각 계층마다 [400,800] 크기의 가중치 행렬과 [800]크기의 편향벡터,
        # 소프트 맥스 계층의 [200,10000] 크기의 가중치 행렬과 [10000]크기의 편향 벡터
        
        grads, _ = tf.clip_by_global_norm(tf.gradients(self.cost, tvars),
                                          config.max_grad_norm)
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        self.train_op = optimizer.apply_gradients(zip(grads, tvars))
        
        # ??? 한 계층의 p_f, p_i, p_j, p_d를 구하는 ,,,???
        
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

In [ ]:
# 에포크를 처리할 함수를 만듭니다.
# 이렇게 만든 텐서플로 모델을 실행시키고 셀에서 나온 은닉 상태와 셀 상태 데이터를 업데이트 하는 역할을 합니다.
# 위 config 클래스의 max_max_epoch 만큼 반복을 진행하여 오차를 최소화 하고자 함. 

def run_epoch(session, m, data, is_training=False): # 텐서플로의 세션 객체, PTBModel 객체, 학습 데이터 전달받음
    """Runs the model on the given data."""
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0
    
    # session.run() 의 매개변수는 연산할 그래프들과 그래프에 주입할 텐서 리스트
    # 연산할 텐서는 PTBModel 클래스의 cost, train_op, fianl_state 튜플에 담겨있는 두 계층의 셀상태와 은닉상태 데이터
    eval_op = m.train_op if is_training else tf.no_op()
    
    # initial_state는 2x20x200 크기의 튜플입니다. 셀의 상태 데이터가 저장되어있고, 모델을 학습시킨 후
    # 다음 데이터를 주입할 때 셀의 상태 데이터로 사용된다. batch 사이즈가 20이므로, 각 셀마다 20개의 상태 데이터 저장해야함
    # 따라서 initial_state는 2개의 계층, 2종류의 상태가 반영된 [2,2] 크기의 튜플. 
    # 튜플의 각 요소는 20개의 배치, 200개의 셀이 반영된 [20,200] 크기의 텐서
    
    # 그래프에 주입할 텐서를 준비
    state_list = []
    
    # 완전 처음에는 initial_state가 평가가 되지 않은 상태 이므로 eval메소드를 사용하여 feed_dict에 주입할 수 있도록
    # state_list에 담아놓는다
    for c, h in m.initial_state:
        state_list.extend([c.eval(), h.eval()])
    
    # initial_state를 순회하면서 셀 상태와 은닉 상태에 각각 state_list의 값을 할당
    
    
    
    ptb_iter = reader.ptb_iterator(data, m.batch_size, m.num_steps)
    
    for step, (x, y) in enumerate(ptb_iter):
       
        fetch_list = [m.cost]
        # final_state 튜플에 담겨있는 상태를 꺼내어 fetch_list에 담습니다. 
        for c, h in m.final_state:
            fetch_list.extend([c, h])
        fetch_list.append(eval_op)
        
        # 이전 스텝에서 구해진 state_list가 feed_dict로 주입됩니다.
        # feed_dict에 모델의 입력데이터와 목표 데이터를 할당하고
        feed_dict = {m.input_data: x, m.targets: y}
        
        # 하나의 batch 연산이 끝나면 fetch_list에 들어있는 final_state를 다음번 batch의 initial_state로 주입하기 위해
        # state_list 변수로 전달
        for i in range(len(m.initial_state)):
            c, h = m.initial_state[i]
            feed_dict[c], feed_dict[h] = state_list[i*2:(i+1)*2]
        
        # fetch_list에 담긴 final_state의 결과를 state_list로 전달 받습니다.
        cost, *state_list, _ = session.run(fetch_list, feed_dict)

        costs += cost
        iters += m.num_steps

        if is_training and step % (epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                    (step * 1.0 / epoch_size, np.exp(costs / iters),
                     iters * m.batch_size / (time.time() - start_time)))

    return np.exp(costs / iters)

In [ ]:
raw_data = reader.ptb_raw_data('simple-examples/data')
train_data, valid_data, test_data, _ = raw_data

In [ ]:
#train_data, valid_data, test_data 는 단어를 숫자로 바꾼 리스트입니다.
#가장 많이 나온 단어 순으로 0번 부터 시작하여 10000번 까지의 번호를 가지고 있습니다.
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)

    # 학습과 검증, 테스트를 위한 모델을 만듭니다.
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel(config, is_training=True)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel(config)
        mtest = PTBModel(eval_config)
        
    tf.global_variables_initializer().run()
    
    for i in range(config.max_max_epoch):
        # lr_decay는 반복속도를 조절해 주는 역할을 합니다.
        lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
        m.assign_lr(session, config.learning_rate * lr_decay)
        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        perplexity = run_epoch(session, m, train_data, is_training=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, perplexity))

        perplexity = run_epoch(session, mvalid, valid_data)
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, perplexity))

    perplexity = run_epoch(session, mtest, test_data)
    print("Test Perplexity: %.3f" % perplexity)